In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import keras_tuner as kt
from tensorflow_addons.metrics import RSquare
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

c:\Users\anton_glfk00f\miniconda3\envs\tf\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


##### Utility functions

In [4]:
def input_mix_dtypes(df, binary_feature_names=[], 
                     categorical_feature_names=[]):
    '''
    It returns dictionary of tensors. It creates the "placeholder" for 
    input tensor from DataFrame.
    '''
    inputs = {}
    for name, column in df.items():
        # setting the right dtype
        if type(column[0]) == str:
            dtype = tf.string
        elif (name in categorical_feature_names or
                name in binary_feature_names):
            dtype = tf.int64 # categorical integer
        else:
            dtype = tf.float32
        inputs[name] = tf.keras.Input(
            shape=(), # scalar input
            name=name, 
            dtype=dtype 
        )
    return inputs

In [5]:
def binary_preprocessor(preprocessed_list, inputs, binary_feature_names):
    '''
    It appends the binary preprocessors tensors to list of preprocessors. No transformation
    performed other than adding a new axia and casting
    '''
    for name in binary_feature_names:
        inp_bin = inputs[name]
        inp_bin_nw_axis = inp_bin[:, tf.newaxis]
        inp_bin_float = tf.cast(inp_bin_nw_axis, tf.float32)
        preprocessed_list.append(inp_bin_float) #the tensor, not the input dict
    return None

In [6]:
# numeric processing
def stack_dict(dict, function=tf.stack):
    '''
    take a dict of pd.Series or a dict of tensors and turns it into a tensor. Used for
    formating the input for adapt operation.
    '''
    values = [] #list of tensors
    for key in sorted(dict.keys()):
        # sort to ensure later matching to preserve order
        input_float = tf.cast(dict[key], tf.float32) 
        values.append(input_float)
    tensor_stacked = function( # stacks the list of tensors into a single one
                    values,
                    axis=-1 # last axis direction, in this case 1?
                    ) 
    return tensor_stacked

def numeric_preprocessor(df_adapter, stack_dict, preprocessed_list, 
                         inputs, numeric_feature_names):
    '''
    It appends the numeric preprocessor tensor to list of preprocessors. Normalization layer
    is used for trasnformation. 
    '''
    # adapting the normalizer
    df_adapter = df_adapter[numeric_feature_names]
    normalizer = tf.keras.layers.Normalization(axis=-1)
    tensor_dfcolumns = stack_dict(dict(df_adapter))
    normalizer.adapt(tensor_dfcolumns) # equivalent to fit

    #input for numeric only
    inputs_numeric = {}
    for name in numeric_feature_names:
        inputs_numeric[name] = inputs[name]

    tensor_numeric_inputs = stack_dict(inputs_numeric)
    numeric_normalized = normalizer(tensor_numeric_inputs) # similar to transform

    preprocessed_list.append(numeric_normalized)
    return None

In [7]:
# categorical processing
def categorical_preprocessor(df_adapter, preprocessed_list, 
                             inputs, categorical_feature_names):
    '''
    It appends the categorical preprocessors tensors to list of preprocessors. It adds a 
    a new axis to each input tensor and then adds a lookup layer.
    '''
    for name in categorical_feature_names:
        vocab = sorted(set(df_adapter[name]))

        # create lookup layers
        if type(vocab[0]) is str:
            lookup = tf.keras.layers.StringLookup(vocabulary=vocab, 
                                                  output_mode='one_hot')
        else:
            lookup = tf.keras.layers.IntegerLookup(vocabulary=vocab, 
                                                   output_mode='one_hot')
        
        input = inputs[name][:,tf.newaxis] # for compatibility
        one_hot_tensor = lookup(input)
        preprocessed_list.append(one_hot_tensor)
    return None

In [22]:
def preprocessor_tensor(numeric_feature_names,
                       binary_feature_names, 
                       categorical_feature_names,
                       X_train,
                       df, #consider all categories
                       inputs):
    '''
    It returns the preprocessor model. Take the names of inputs for preserving order
    '''
    preprocessed_list = [] # list of input, preprocessed tensors

    binary_preprocessor(preprocessed_list, inputs, binary_feature_names)
    numeric_preprocessor(X_train,  stack_dict, preprocessed_list, 
                         inputs, numeric_feature_names)
    categorical_preprocessor(df, preprocessed_list, inputs, 
                             categorical_feature_names)

    preprocessed_result = tf.concat(preprocessed_list, axis=-1) # preprocessed tensor

    return preprocessed_result

In [47]:
def plot_metrics(metrics, history, start_epoch, end_epoch):
    """
    It plots the defined metrics per epoch
    """

    num_metrics = len(metrics)
    num_rows = (num_metrics + 1) // 2  # Compute the number of rows based on the number of metrics
    
    # Create the grid of subplots
    fig, axs = plt.subplots(num_rows, 2, figsize=(10, 8))

    # Flatten the subplots array for easy iteration
    axs = axs.flatten()

    # Iterate over the metrics and corresponding subplots
    for i, metric in enumerate(metrics):
        # Get the range of epochs to plot
        metric_values = history.history[metric][start_epoch:end_epoch]
        val_metric_values = history.history['val_' + metric][start_epoch:end_epoch]
        
        # Plot train metric with updated x-axis
        axs[i].plot(range(start_epoch, end_epoch), metric_values)
        
        # Plot validation metric with updated x-axis
        axs[i].plot(range(start_epoch, end_epoch), val_metric_values)
        
        # Set subplot title and labels
        title = metric.replace('_', ' ')
        axs[i].set_title(title)
        axs[i].set_ylabel('Loss')
        axs[i].set_xlabel('Epoch')
        axs[i].legend(['Train', 'Validation'], loc='upper right')

    # Adjust spacing between subplots
    plt.tight_layout()

    # Display the plot
    plt.show()


#### Model building

In [27]:
# setting the constants
SHUFFLE_BUFFER = 500
BATCH_SIZE = 32
RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [10]:
# importing and splitting
data_path = '../data/real_state_data_for_model.csv'
df = pd.read_csv(data_path)

# defining the features and variables
X = df.drop('precio', axis=1)
y = df['precio']
y_stratify = X[['tipo_de_propiedad']] # keep proportions for this variable

# train-val-test splitting
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size = 0.30, 
    random_state=RANDOM_SEED,
    stratify=y_stratify,
)

y_stratify = X_test[['tipo_de_propiedad']]
X_val, X_test, y_val, y_test = train_test_split(
    X_test, 
    y_test, 
    test_size = 0.5, 
    random_state=RANDOM_SEED,
    stratify=y_stratify,
)

# variables for transformations 
numeric_feature_names = ['no_ba単os', 'terreno_m2', 'a単o_constr',
                        'no_dormitorios', 'area_constr_m2', 'latitud', 
                        'longitud', 'estacionamientos', 'ambientes']
categorical_feature_names = ['tipo_de_propiedad', 'ciudad', 'zona']
binary_feature_names = []


In [51]:
# baseline model

# create dict of input tensors 
inputs = input_mix_dtypes(X_train, binary_feature_names, 
                          categorical_feature_names)

# create preprocessor, now returns a tensor
preprocessed_tensor = preprocessor_tensor(numeric_feature_names, binary_feature_names, 
                                  categorical_feature_names, X_train, X, inputs)

body = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu',),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1),
])

#preprocessed_tensor = preprocessor(inputs)
result_tensor = body(preprocessed_tensor)
model = tf.keras.Model(inputs, result_tensor) # "double considering"  inputs causes the "error":

model.compile(optimizer='adam',
    loss=tf.keras.losses.MeanSquaredError(name='mean_squared_error'),
        metrics=[
        RSquare(name='RSquare'), #all metrics are normalized
        tf.keras.losses.MeanSquaredError(name="mean_squared_error"),
        tf.keras.metrics.MeanAbsoluteError(name='mean_absolute_error'),
        tf.keras.metrics.RootMeanSquaredError(name='root_mean_squared_error'),
        tf.keras.metrics.MeanAbsolutePercentageError(name="mean_absolute_percentage_error"),
    ]
)

history_base_model = model.fit(dict(X_train), y_train, epochs=10, 
                    batch_size=BATCH_SIZE, verbose=0,
                    validation_data=(dict(X_val), y_val),
                    )

metrics = ['RSquare', 'mean_absolute_error', 'root_mean_squared_error', 
           'mean_absolute_percentage_error', 'loss', 'mean_squared_error']
           # loss and mean_squared_error are the same
           
#print(plot_metrics(metrics, history_base_model, 0, 10))

# visualizing the model
#tf.keras.utils.plot_model(model, rankdir="LR", show_shapes=True, expand_nested=True)

#### Hiperparameter tuning

In [35]:
def compiled_model(hp,
                   learning_rate,
                   layers,
                   activation,
                   ): # if input_mix_dtypes, preprocessor_model are defined it works

    # create dict of input tensors 
    inputs = input_mix_dtypes(X_train, binary_feature_names, 
                            categorical_feature_names)

    # create preprocessor
    preprocessed_tensor = preprocessor_tensor(numeric_feature_names, binary_feature_names, 
                                    categorical_feature_names, X_train, X, inputs)

    # building the body
    body = tf.keras.Sequential()
    for i in range(layers):
        neurons = hp.Int(f"units_{i}", min_value=30, max_value=150, step=10)
        regularization_kernel = hp.Float(f"r_k{i}", min_value=1e-2, 
                                        max_value=100, sampling="log")
        regularization_bias = hp.Float(f"r_b{i}", min_value=1e-2, 
                                        max_value=100, sampling="log")
        body.add(tf.keras.layers.Dense(
            neurons, 
            activation=activation,
            kernel_regularizer=tf.keras.regularizers.L2(regularization_kernel),
            bias_regularizer=tf.keras.regularizers.L2(regularization_bias),
            )
        )
    body.add(tf.keras.layers.Dense(1))

    # building the final model
    result_tensor = body(preprocessed_tensor)
    model = tf.keras.Model(inputs, result_tensor)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.MeanSquaredError(name='mean_squared_error'),
        metrics=[
            RSquare(name='RSquare'),
            tf.keras.metrics.MeanAbsoluteError(name='mean_absolute_error'),
            tf.keras.metrics.RootMeanSquaredError(name='root_mean_squared_error'),
            tf.keras.metrics.MeanAbsolutePercentageError(name='mean_absolute_percentage_error'),
        ]
    )
    return model

def build_model(hp):
    activation = hp.Choice("activation", ["selu"]) #["relu", "tanh", "selu"])
    layers = 5 #hp.Int("layers", min_value=1, max_value=5, step=1)
    learning_rate = hp.Float("lr", min_value=1e-5, max_value=0.1, sampling="log")

    # call existing model-building code with the hyperparameter values.
    model = compiled_model(hp,
                   learning_rate,
                   layers,
                   activation,
                   )
    return model


tuner = kt.Hyperband(
                     build_model,
                     objective= kt.Objective('val_root_mean_squared_error', direction='min'),
                     max_epochs=150,
                     factor=10,
                     directory='my_dir',
                     project_name='intro_to_kt',
                     overwrite=True
                     )

# hyperparameters search
tuner.search(dict(X_train), y_train, epochs=100, batch_size=BATCH_SIZE,
            validation_data=(dict(X_val), y_val))

Trial 176 Complete [00h 01m 49s]
val_root_mean_squared_error: 15271.763671875

Best val_root_mean_squared_error So Far: 10000.2802734375
Total elapsed time: 00h 31m 00s
INFO:tensorflow:Oracle triggered exit


In [52]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0] # see if gpt is right

# optimal epochs search
model = tuner.hypermodel.build(best_hps)
history = model.fit(dict(X_train), y_train, epochs=100, 
                    validation_data=(dict(X_val), y_val), verbose=0)

val_loss_per_epoch = history.history['val_loss']

best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

metrics_hipersearch = ['mean_absolute_percentage_error', 'mean_absolute_error',
           'root_mean_squared_error', 'loss', 'RSquare']
plot_metrics(metrics_hipersearch, history, 10, best_epoch)

Best epoch: 89


In [55]:
# get final model
best_model = tuner.hypermodel.build(best_hps)
best_epoch = 50 # adjusting from plots
history = best_model.fit(dict(X_train), y_train, epochs=best_epoch, validation_data=(dict(X_val), y_val))

Epoch 1/50
32/32 [==============================] - 1s 19ms/step - loss: 7444844544.0000 - RSquare: -8.3139 - mean_absolute_error: 75807.0234 - root_mean_squared_error: 86283.2812 - mean_absolute_percentage_error: 79.7162 - val_loss: 1159655296.0000 - val_RSquare: -0.3190 - val_mean_absolute_error: 28082.9512 - val_root_mean_squared_error: 34052.3945 - val_mean_absolute_percentage_error: 33.8117
Epoch 2/50
32/32 [==============================] - 0s 10ms/step - loss: 676935872.0000 - RSquare: 0.1533 - mean_absolute_error: 20186.0078 - root_mean_squared_error: 26015.7422 - mean_absolute_percentage_error: 24.3478 - val_loss: 340150176.0000 - val_RSquare: 0.6132 - val_mean_absolute_error: 13412.7959 - val_root_mean_squared_error: 18439.5020 - val_mean_absolute_percentage_error: 18.2649
Epoch 3/50
32/32 [==============================] - 0s 9ms/step - loss: 327016640.0000 - RSquare: 0.5911 - mean_absolute_error: 13996.5742 - root_mean_squared_error: 18079.7578 - mean_absolute_percentage_er

In [56]:
# evaluating the final model
print('Training set')
print(best_model.evaluate(dict(X_train),  y_train, verbose=2))
print('r2 score2 training', r2_score(y_train, best_model.predict(dict(X_train))))
print('Validation set')
print(best_model.evaluate(dict(X_val),  y_val, verbose=2))
print('r2 score2 validation', r2_score(y_val, best_model.predict(dict(X_val))))
print('Test set')
print(best_model.evaluate(dict(X_test),  y_test, verbose=2))
print('r2 score2 test', r2_score(y_test, best_model.predict(dict(X_test))))

plot_metrics(metrics_hipersearch, history, 20, best_epoch)

# visualizing model structure
tf.keras.utils.plot_model(best_model, rankdir="LR",expand_nested=True)

Training set
32/32 - 0s - loss: 55108284.0000 - RSquare: 0.9313 - mean_absolute_error: 5634.7676 - root_mean_squared_error: 7408.5239 - mean_absolute_percentage_error: 6.3655 - 170ms/epoch - 5ms/step
[55108284.0, 0.9313337206840515, 5634.767578125, 7408.52392578125, 6.365549564361572]
32/32 [==============================] - 0s 4ms/step
r2 score2 training 0.9313337542848397
Validation set
7/7 - 0s - loss: 139531776.0000 - RSquare: 0.8415 - mean_absolute_error: 7526.8740 - root_mean_squared_error: 11802.9531 - mean_absolute_percentage_error: 8.9695 - 63ms/epoch - 9ms/step
[139531776.0, 0.8415369987487793, 7526.8740234375, 11802.953125, 8.969470024108887]
7/7 [==============================] - 0s 3ms/step
r2 score2 validation 0.8415367038228931
Test set
7/7 - 0s - loss: 202412528.0000 - RSquare: 0.7517 - mean_absolute_error: 8537.5430 - root_mean_squared_error: 14219.3682 - mean_absolute_percentage_error: 10.1542 - 54ms/epoch - 8ms/step
[202412528.0, 0.7517449259757996, 8537.54296875, 14

#### Exporting the model

In [96]:
# exporting and importing the model
print(best_model.predict(dict(X_train.iloc[:1])))
best_model.save('model.h5')
model_1 = tf.keras.models.load_model('model.h5')
print(model_1.predict(dict(X_train.iloc[:1])))

1/1 [==============================] - 0s 24ms/step
[[80905.586]]
1/1 [==============================] - 0s 117ms/step
[[80905.586]]


#### Debugging

In [97]:
data = {
    'ambientes': [7],
    'no_ba単os': [2],
    'terreno_m2': [393],
    'a単o_constr': [2013],
    'no_dormitorios': [3],
    'area_constr_m2': [100.0],
    'estacionamientos': [4],
    'latitud': [-17.643187],
    'longitud': [-63.173777],
    'tipo_de_propiedad': ['Casa'],
    'ciudad': ['Santa Cruz de la Sierra'],
    'zona': ['Norte']
}
#imput_data: dataframe
data_1 = pd.DataFrame(data)
print(model_1.predict(dict(data_1)))


1/1 [==============================] - 0s 123ms/step
[[80372.24]]


In [ ]:
# summary of models
#tuner.results_summary()
#tuner.search_space_summary()
#print(tuner.get_best_hyperparameters()[0].values)
#print(best_model.get_layer('sequential').summary())

In [ ]:
# other command to get best model
#best_model = tuner.get_best_models()[0]

In [ ]:
# for ploting outside jypyter
import pydot
from tensorflow.keras.utils import model_to_dot
dot = model_to_dot(model)
graph = pydot.graph_from_dot_data(dot.to_string())[0]
graph.write_png('model.png')